In [ ]:
import folium
from folium import IFrame
from IPython.display import display, IFrame as IPythonIFrame
from branca.element import Figure, CssLink, JavascriptLink

#convert DMS to Decimal Degrees
def dms_to_dd(dms_str):
    dms_str = dms_str.replace(',', '.')  # Replace commas with dots
    degrees, minutes, seconds, direction = 0, 0, 0, ''
    
    dms_str = dms_str.strip()
    if '°' in dms_str and "'" in dms_str and '"' in dms_str:
        degrees, rest = dms_str.split('°')
        minutes, rest = rest.split("'")
        seconds, direction = rest.split('"')
    elif '°' in dms_str and "'" in dms_str:
        degrees, rest = dms_str.split('°')
        minutes, direction = rest.split("'")
    elif '°' in dms_str:
        degrees, direction = dms_str.split('°')
    
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60)
    if direction in ['S', 'W']:
        dd *= -1
    return dd


cell_towers = [
    {"site": "Cell Tower 1", "lat_dms": "X° X'XX.X\"S", "lng_dms": "XX° X'XX.X\"W", "height": 30.4}
]

#DMS to DD
for tower in cell_towers:
    tower["lat_dd"] = dms_to_dd(tower["lat_dms"])
    tower["lng_dd"] = dms_to_dd(tower["lng_dms"])

m = folium.Map(location=[-3.13, -60.02], zoom_start=7)
#CSS and JavaScript link for jQuery UI tabs
figure = Figure()
figure.add_child(CssLink("https://code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css"))
figure.add_child(JavascriptLink("https://code.jquery.com/jquery-1.12.4.js"))
figure.add_child(JavascriptLink("https://code.jquery.com/ui/1.12.1/jquery-ui.js"))
figure.add_child(m)

#markers
for tower in cell_towers:
    html = f"""
    <div>
        <p>Site: {tower['site']}<br>Height: {tower['height']}m</p>
    </div>
    <div id="tabs-{tower['site']}">
      <ul style="list-style-type:none;">
        <li><a href="#tabs-1">Tab 1</a></li>
        <li><a href="#tabs-2">Tab 2</a></li>
        <li><a href="#tabs-3">Tab 3</a></li>
      </ul>
      <div id="tabs-1">
        <p>Additional Info 1</p>
      </div>
      <div id="tabs-2">
        <p>Additional Info 2</p>
      </div>
      <div id="tabs-3">
        <p>Additional Info 3</p>
      </div>
    </div>
    <script>
    $( function() {{
        $( "#tabs-{tower['site']}" ).tabs();
    }} );
    </script>
    """
    iframe = IFrame(html, width=250, height=200)
    popup = folium.Popup(iframe, max_width=300)
    icon = folium.Icon(icon='signal', prefix='fa')
    folium.Marker(
        location=[tower["lat_dd"], tower["lng_dd"]],
        popup=popup,
        icon=icon
    ).add_to(m)

for i in range(len(cell_towers) - 1):
    point1 = [cell_towers[i]["lat_dd"], cell_towers[i]["lng_dd"]]
    point2 = [cell_towers[i+1]["lat_dd"], cell_towers[i+1]["lng_dd"]]
    folium.PolyLine(locations=[point1, point2], color='orange').add_to(m)

m.save('/map.html')
display(IFrame('/map.html', width=700, height=500))